In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import seaborn as sns

import bayesflow as bf
from functools import partial
from IPython.display import HTML

# Amortized Posterior Estimation for Simple Agent-Based Model
**Vicsek Model in BayesFlow** 

In this notebook, we demonstrate amortized posterior estimation for a simple agent-based model (ABM). ABM is of interest because it is inherently hierarchical, but the interaction between agents does not always lead to tractable mathematical formulation. In addition, the resulting formulation can be either vague, involving little to no equations, or extremely complex, involving a large amount of parameters and equations, making meaningful inference difficult. 

Neural surrogates have proven to be one of the promising approaches for extracting model parameters for ABM. As an example, we use BayesFlow for the posterior estimation for a simple ABM: the [Vicsek model](https://en.wikipedia.org/wiki/Vicsek_model). The model, which characterizes the dynamic of collective motion, has found numerous applications from simulating active Brownian particles, from modeling social dynamics.

The model is formulated as follows:

\begin{align}
    \theta_{j, t} &= \langle \theta_{i, t}\rangle_{|r_j - r_i| < r} + \eta_{j,t-1}, \\
    \mathbf{x}_{j,t} &= \mathbf{x}_{j,t-1} + v \mathrm{d}t
    \begin{bmatrix}
        \cos \theta_{j, t} \\
        \sin \theta_{j, t}
    \end{bmatrix},
\end{align}

where

* $\theta_{j, t}, \theta_{i, t}$ is the heading direction of agent $j$ and $i$ at a given time $t$. Here, $i$ is the neighboring agent of $j$ within a perception distance $r$;
* $\eta_{j, t-1} \sim \mathcal{U}(-\mu_j, \mu_j)$ is a rotational noise for agent $j$. In addition to uniform sampling, this noise can also be sampled from  Gaussian or von Mises distributions;
* $\mathbf{x}_{j, t}$ is the position of agent $j$ as a given time $t$; and
* $v$ is the speed of agent $j$.

In [5]:
# Constants
NUM_AGENTS = 49     # Number of agents
TIME_STEPS = 1000   # Number of time steps
BOUND_SIZE = 10     # Size of the square domain

# Parameter names
PARAM_NAMES = [r"$\mu_j$", r"$r_j$", r"$v_j$"]

### Hyperpriors and Priors



In [ ]:
def prior_fun(rng=None):

    if rng is None:
        rng = np.random.default_rng()
    
    

In [ ]:
# Parameters
num_agents = 100
L = 10.0  # size of the square domain
v = 0.03  # speed of agents
eta = 0.1  # noise amplitude
r = 1.0  # interaction radius
time_steps = 300

# Initialize agents with random positions and directions
positions = np.random.rand(num_agents, 2) * L
angles = np.random.rand(num_agents) * 2 * np.pi
velocities = np.vstack((np.cos(angles), np.sin(angles))).T * v

def update_positions(positions, velocities, L):
    positions += velocities
    positions %= L  # Periodic boundary conditions
    return positions

def update_velocities(positions, velocities, eta, r, L):
    new_velocities = np.zeros_like(velocities)
    for i, pos in enumerate(positions):
        # Find neighbors within radius r
        distances = np.linalg.norm(positions - pos, axis=1)
        neighbors = np.where(distances < r)[0]

        # Compute average direction
        avg_direction = np.arctan2(np.mean(np.sin(angles[neighbors])), np.mean(np.cos(angles[neighbors])))
        new_angle = avg_direction + eta * (np.random.rand() - 0.5)
        new_velocities[i] = np.array([np.cos(new_angle), np.sin(new_angle)]) * v

    return new_velocities

# Simulation


for _ in range(time_steps):
    velocities = update_velocities(positions, velocities, eta, r, L)
    positions = update_positions(positions, velocities, L)

    # Visualization (optional)
    plt.clf()
    plt.quiver(positions[:, 0], positions[:, 1], velocities[:, 0], velocities[:, 1])
    plt.xlim(0, L)
    plt.ylim(0, L)
    plt.pause(0.01)

plt.show()